In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import html
import matplotlib as plt
import re
from itertools import combinations
import time
pd.set_option('display.max_columns', None)

In [2]:
historic_df=pd.read_csv('cleaned_historic_inspections.csv')

In [3]:
historic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14428 entries, 0 to 14427
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   objectid                                  14428 non-null  int64  
 1   globalid                                  14428 non-null  object 
 2   Seating Interest (Sidewalk/Roadway/Both)  14428 non-null  object 
 3   Restaurant Name                           14428 non-null  object 
 4   Legal Business Name                       14428 non-null  object 
 5   Doing Business As (DBA)                   14427 non-null  object 
 6   Building Number                           14329 non-null  object 
 7   Street                                    14428 non-null  object 
 8   Borough                                   14428 non-null  object 
 9   Postcode                                  14428 non-null  int64  
 10  Business Address                  

In [4]:
historic_df['Doing Business As (DBA)'].fillna(historic_df['Restaurant Name'], inplace=True)
historic_df['Column Of Interest'] = historic_df['Restaurant Name'] + " " + historic_df['Street']
historic_df['Global Restaurant ID'] = ''
historic_df['Latest Inspection Row'] = ''
historic_df['BIN'] = historic_df['BIN'].astype(str).replace('nan', 'undefined').replace('nan', 'undefined').fillna('undefined')

In [5]:
historic_df['BIN'].value_counts()

undefined    1402
3000000.0      29
4000000.0      21
3397861.0      18
3000090.0      17
             ... 
4124422.0       1
2013298.0       1
1035447.0       1
4311574.0       1
1025423.0       1
Name: BIN, Length: 8960, dtype: int64

In [6]:
#Function to fill Global Restaurant ID
global_restaurant_id_counter = 0
def assign_global_restaurant_id(df, threshold=85):
    global global_restaurant_id_counter
    # Filter rows where 'Global Restaurant ID' is NaN
    filtered_df = df[df['Global Restaurant ID'] == '']

    # Iterate through rows in the group
    for bin_value in filtered_df['BIN'].unique():
        # Group rows by BIN
        bin_group = filtered_df[filtered_df['BIN'] == bin_value]

        # Iterate through rows in the BIN group
        for index, row in bin_group.iterrows():
            print("Current bin value", bin_value)
            print("Current index", index)
            # Check if the 'Column of Interest' values are similar
            if pd.isna(filtered_df.loc[index, 'Global Restaurant ID']) or filtered_df.loc[index, 'Global Restaurant ID'] == '':
                # Check if the 'Column of Interest' values are similar with other rows in the group
                match = bin_group[bin_group.index != index]['Column Of Interest'].apply(lambda x: fuzz.ratio(row['Column Of Interest'], x) > threshold)
                print("The sum of matches is:", match.sum())
                print(match)

                # If there are matches, assign the same global restaurant ID to the current row and matching rows
                if match.any():
                    global_id = f"{global_restaurant_id_counter:08d}"
                    filtered_df.at[index, 'Global Restaurant ID'] = global_id
                    filtered_df.loc[match.index[match==True], 'Global Restaurant ID'] = global_id
                    global_restaurant_id_counter += 1
                else:
                    # If no match, assign the next global restaurant ID to the single row
                    filtered_df.at[index, 'Global Restaurant ID'] = f"{global_restaurant_id_counter:08d}"
                    global_restaurant_id_counter += 1
            else:
                print("Row already has value")

    return filtered_df


    return filtered_df
    # Merge the filtered_df back to the original df based on unique_column and copy the 'Global Restaurant ID' values
#df = pd.merge(df, filtered_df[[unique_column, 'Global Restaurant ID']], on=unique_column, how='left', suffixes=('', '_filtered'))
#df['Global Restaurant ID'] = df['Global Restaurant ID_filtered'].combine_first(df['Global Restaurant ID'])
#df.drop(columns=['Global Restaurant ID_filtered'], inplace=True)

#return df
result_df = assign_global_restaurant_id(historic_df,threshold=85)
result_df

Current bin value 1049941.0
Current index 0
The sum of matches is: 2
1252    True
7526    True
Name: Column Of Interest, dtype: bool
Current bin value 1049941.0
Current index 1252
Row already has value
Current bin value 1049941.0
Current index 7526
Row already has value
Current bin value 1054995.0
Current index 1
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1043896.0
Current index 2
The sum of matches is: 0
4603    False
Name: Column Of Interest, dtype: bool
Current bin value 1043896.0
Current index 4603
The sum of matches is: 0
2    False
Name: Column Of Interest, dtype: bool
Current bin value 1014129.0
Current index 3
The sum of matches is: 1
3019    True
Name: Column Of Interest, dtype: bool
Current bin value 1014129.0
Current index 3019
Row already has value
Current bin value 1005393.0
Current index 4
The sum of matches is: 1
1156    True
Name: Column Of Interest, dtype: bool
Current bin value 1005393.0
Current index 1156
Row alread

The sum of matches is: 2
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 338
The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 343
The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 360
The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277 

The sum of matches is: 2
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 651
The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 658
The sum of matches is: 1
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 668
The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277 

The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 943
The sum of matches is: 1
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 944
The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 957
The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277 

The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 1452
The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 1454
The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
14277    False
14288    False
14417    False
14419    False
Name: Column Of Interest, Length: 1401, dtype: bool
Current bin value undefined
Current index 1456
The sum of matches is: 0
6        False
9        False
11       False
43       False
44       False
         ...  
14259    False
142

...

Current bin value 3328782.0
Current index 13084
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 4606129.0
Current index 13089
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1022629.0
Current index 13091
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 3338048.0
Current index 13092
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1001725.0
Current index 13094
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1050371.0
Current index 13095
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1011045.0
Current index 13097
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1085734.0
Current index 13099
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)


Current bin value 1019848.0
Current index 13438
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1003865.0
Current index 13440
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 3022098.0
Current index 13442
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1057908.0
Current index 13443
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1065082.0
Current index 13444
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1051454.0
Current index 13446
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1008692.0
Current index 13447
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1011738.0
Current index 13448
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)


Current bin value 1057434.0
Current index 13911
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 4194283.0
Current index 13914
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 3002769.0
Current index 13917
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1009923.0
Current index 13918
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1047964.0
Current index 13921
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 3088255.0
Current index 13922
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1023727.0
Current index 13926
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1088724.0
Current index 13928
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)


Current bin value 3072416.0
Current index 14372
Row already has value
Current bin value 1084928.0
Current index 14342
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1008044.0
Current index 14348
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1003283.0
Current index 14352
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 1005840.0
Current index 14355
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 3127587.0
Current index 14357
The sum of matches is: 0
Series([], Name: Column Of Interest, dtype: object)
Current bin value 3019807.0
Current index 14358
The sum of matches is: 1
14362    True
Name: Column Of Interest, dtype: bool
Current bin value 3019807.0
Current index 14362
Row already has value
Current bin value 3325603.0
Current index 14361
The sum of matches is: 0
Series([], Name: Column Of Interest, 

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Column Of Interest,Global Restaurant ID,Latest Inspection Row
0,13610,FD87ABAA-860E-4762-845D-8F0403D0246B,Roadway,Madame Bonte,Beanhouse LLC,Madame Bonte,318,East 84th Street,Manhattan,10028,"318 East 84th Street, Manhattan, NY",50114174,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,yes,1346965.0,TW,no,NaN,yes,06/09/2022 04:26:00 PM,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville,Madame Bonte East 84th Street,00000000,
1,5900,3B07E4C0-07B7-4079-8333-64446CC3EE03,Sidewalk,Seasoned Vegan,"Seasoned Vegan, LLC","Seasoned Vegan, LLC",55,St. Nicholas Avenue,Manhattan,10026,"55 St. Nicholas Avenue, Manhattan, NY",50003337,47.0,3.0,141.0,NaN,NaN,NaN,yes,no,yes,1289526.0,RW,no,NaN,yes,06/26/2020 08:38:00 PM,40.800500,-73.952507,10.0,9.0,216.0,1054995.0,1.018220e+09,Central Harlem South,Seasoned Vegan St. Nicholas Avenue,00000001,
2,13018,137C575D-DC14-4F9D-83D9-A3FFE513B3B8,Sidewalk,Americas Cafe & Grill,68th Grill Inc.,Americas Cafe & Grill,1159,3rd Avenue,Manhattan,10065,"1159 3rd Avenue, Manhattan, NY",40797684,8.0,8.0,64.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,10/22/2021 11:01:00 AM,40.766845,-73.962708,8.0,4.0,118.0,1043896.0,1.014220e+09,Lenox Hill-Roosevelt Island,Americas Cafe & Grill 3rd Avenue,00000002,
3,11630,15270732-2A78-4C24-89DD-BE8DD916F115,Roadway,Sushi Seki,Seki Inc.,Sushi Seki,208,West 23rd Street,Manhattan,10011,"208 West 23rd Street, Manhattan, NY",50005983,NaN,NaN,NaN,44.0,8.0,352.0,no,yes,yes,0.0,OP,no,NaN,yes,12/14/2020 07:54:00 PM,40.744338,-73.996240,4.0,3.0,91.0,1014129.0,1.007720e+09,Hudson Yards-Chelsea-Flatiron-Union Square,Sushi Seki West 23rd Street,00000004,
4,13137,EF9C8173-91D1-496E-8BD4-B02BEADC2A21,Roadway,Sami & Susu,Amir Nathan,Sami & Susu,190,Orchard Street,Manhattan,10002,"190 Orchard Street, Manhattan, NY",50112624,NaN,NaN,NaN,22.0,8.0,176.0,no,yes,yes,0.0,TW,no,NaN,yes,12/22/2021 12:46:00 PM,40.722124,-73.988160,3.0,1.0,3001.0,1005393.0,1.004120e+09,Chinatown,Sami & Susu Orchard Street,00000005,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14423,9799,108C6784-54DB-480B-8627-CF0495D27A0E,Both,King Of Spades Inc.,King Of Spades Inc.,King Of Spades Inc.,1425,College Point Boulevard,Queens,11356,"1425 College Point Boulevard, Queens, NY",50072483,22.0,4.0,88.0,22.0,8.0,176.0,yes,yes,yes,0.0,OP,no,NaN,yes,08/04/2020 03:27:00 PM,40.784739,-73.845776,7.0,19.0,929.0,4098349.0,4.040850e+09,College Point,King Of Spades Inc. College Point Boulevard,00012038,
14424,8215,7B8EC275-D470-4106-819E-03FC0E1F9717,Both,La Queen Tea House Inc.,La Queen Tea House Inc.,La Queen Tea House Inc.,752B,61st Street,Brooklyn,11220,"752B 61st Street, Brooklyn, NY",50105060,26.0,7.0,182.0,26.0,8.0,208.0,yes,yes,no,NaN,NaN,no,NaN,yes,07/13/2020 03:53:00 PM,40.635543,-74.011220,7.0,38.0,118.0,3143885.0,3.057940e+09,Sunset Park East,La Queen Tea House Inc. 61st Street,00011494,
14425,2895,476F88D2-D3E8-4E83-8686-8BAC2F752CF5,Sidewalk,Aahar Indina Cuisine,Vidhan Bhatt Inc.,Vidhan Bhatt Inc.,10,Murray Street,Manhattan,10007,"10 Murray Street, Manhattan, NY",50049997,25.0,14.0,350.0,NaN,NaN,NaN,yes,no,yes,1298882.0,RW,no,NaN,yes,06/21/2020 01:46:00 PM,40.713298,-74.007773,1.0,1.0,21.0,1001399.0,1.001240e+09,SoHo-TriBeCa-Civic Center-Little Italy,Aahar Indina Cuis

In [7]:
result_df['Global Restaurant ID'].value_counts()

00002253    8
00003640    7
00004557    6
00003164    6
00007509    5
           ..
00001331    1
00006398    1
00000505    1
00006399    1
00012039    1
Name: Global Restaurant ID, Length: 12040, dtype: int64

In [8]:
result_df[result_df['Global Restaurant ID']=='00003713']

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Column Of Interest,Global Restaurant ID,Latest Inspection Row
2037,7254,465C0CC0-C2FB-4D19-8853-6A19A72001DB,Both,Cafe Cello,"Coffield, Inc.","Coffield, Inc.",48,West 46th Street,Manhattan,10036,"48 West 46th Street, Manhattan, NY",40739883,6.0,2.0,12.0,14.0,8.0,112.0,yes,yes,yes,0.0,OP,no,NaN,yes,07/04/2020 01:46:00 PM,40.756682,-73.980599,5.0,4.0,96.0,1034266.0,1.012610e+09,Midtown-Midtown South,Cafe Cello West 46th Street,00003713,


In [9]:
result_df[result_df['Restaurant Name']=='Love And Dough']

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Column Of Interest,Global Restaurant ID,Latest Inspection Row
769,10340,C304F754-FA8F-4809-8583-14786BA65329,Roadway,Love And Dough,Water St Tavern Corp.,Love And Dough,57,Pearl Street,Brooklyn,11201,"57 Pearl Street, Brooklyn, NY",50047483,NaN,NaN,NaN,20.0,8.0,160.0,no,yes,no,NaN,NaN,no,NaN,yes,08/17/2020 04:06:00 PM,40.703056,-73.987582,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Love And Dough Pearl Street,00002253,
2536,12970,A96E9116-F869-4FEA-8BF1-501535D0912C,Roadway,Love And Dough,Water St Tavern Corp.,Love And Dough,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,NaN,NaN,NaN,40.0,8.0,320.0,no,yes,no,NaN,NaN,no,NaN,yes,10/12/2021 03:33:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Love And Dough Jay Street,00002253,
5542,10211,16A76904-73DE-4E1B-8A6A-D17EBDFB2C28,Roadway,Love And Dough,Water Street Tavern Corp.,Water Street Tavern Corp.,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,NaN,NaN,NaN,30.0,8.0,240.0,no,yes,no,NaN,NaN,no,NaN,yes,08/13/2020 02:31:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Love And Dough Jay Street,00002253,
10213,10212,E3C6154D-2C1B-4246-8246-873B55AF0257,Roadway,Love And Dough,Water St Tavern Corp.,Love And Dough,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,NaN,NaN,NaN,30.0,8.0,240.0,no,yes,no,NaN,NaN,no,NaN,yes,08/13/2020 02:34:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Love And Dough Jay Street,00002253,
11721,10341,9711D0D3-B678-485A-897F-D510EA20FDEA,Sidewalk,Love And Dough,Water St Tavern Corp.,Love And Dough,57,Pearl Street,Brooklyn,11201,"57 Pearl Street, Brooklyn, NY",50047483,15.0,4.0,60.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,08/17/2020 04:09:00 PM,40.703056,-73.987582,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Love And Dough Pearl Street,00002253,
12022,1084,542D1FD6-0687-4C61-8772-978BD03C2283,Sidewalk,Love And Dough,Water St Tavern Corp.,Love And Dough,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,10.0,13.0,130.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,06/19/2020 02:57:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Love And Dough Jay Street,00002253,
12073,12852,D5C7E342-D265-4FE9-8187-B80C6A44DD73,Roadway,Love And Dough,Water St Tavern Corp.,Love And Dough,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,NaN,NaN,NaN,20.0,8.0,160.0,no,yes,no,NaN,NaN,no,NaN,yes,09/15/2021 03:01:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Love And Dough Jay Street,00002253,
14062,1177,B4AED4FB-4D6F-4534-802C-6FF3E66BB556,Roadway,Love And Dough,Water St Tavern Corp.,Love And Dough,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,no,NaN,NaN,no,NaN,yes,06/19/2020 03:20:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Love And Dough Jay Street,00002253,


In [10]:
historic_df[historic_df['BIN']=='undefined']

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Column Of Interest,Global Restaurant ID,Latest Inspection Row
6,1426,48F151E9-5E17-41C7-8197-EBB4A5FC5B23,Sidewalk,La Verite Bar Restaurant & Lounge,Same,Same,187-04,187th-4th Hillside Avenue,Queens,11432,"187-04 187th-4th Hillside Avenue, Queens, NY",636471,240.0,48.0,11520.0,NaN,NaN,NaN,yes,no,yes,1126077.0,OP,no,NaN,yes,06/19/2020 04:20:00 PM,NaN,NaN,NaN,NaN,NaN,undefined,NaN,NaN,La Verite Bar Restaurant & Lounge 187th-4th Hi...,,
9,9068,415BD287-CF8E-4A68-8D00-C707F3B2FE62,Both,La Catrina Restaurant Deli,La Catrina Restaurant Mexican Deli Corp.,La Catrina Restaurant Mexican Deli Corp.,2083,Amsterdam W 163rd Street,Manhattan,10032,"2083 Amsterdam W 163rd Street, Manhattan, NY",50093284,10.0,11.0,110.0,10.0,8.0,80.0,yes,yes,no,NaN,NaN,no,NaN,yes,07/21/2020 03:25:00 PM,NaN,NaN,NaN,NaN,NaN,undefined,NaN,NaN,La Catrina Restaurant Deli Amsterdam W 163rd S...,,
11,8903,1FFC0F07-9A96-46BB-8002-37CA32E76790,Sidewalk,Spoons,Spoons Of 13th Avenue Inc.,Spoons Of 13th Avenue Inc.,5001,13th,Brooklyn,11219,"5001 13th, Brooklyn, NY",50071004,26.0,10.0,260.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,07/19/2020 02:45:00 PM,NaN,NaN,NaN,NaN,NaN,undefined,NaN,NaN,Spoons 13th,,
43,2710,E96F1462-EB9F-4C59-8C27-B4BD4ACDC7FA,Both,Pop & Pour,Francisco & Francisco Inc. Dba Pop & Pour,Pop & Pour,200-8,"200th Dyckman Street Frnt 8th, Frnt 8th",Manhattan,10040,"200-8 200th Dyckman Street Frnt 8th, Frnt 8th,...",50056063,18.0,11.0,198.0,18.0,8.0,144.0,yes,yes,yes,1307919.0,OP,no,NaN,yes,06/20/2020 09:51:00 PM,NaN,NaN,NaN,NaN,NaN,undefined,NaN,NaN,"Pop & Pour 200th Dyckman Street Frnt 8th, Frnt...",,
44,8443,A63C2AE1-9833-4804-8D48-FC06A0052A0A,Sidewalk,Sushi Teru,Hui&Yuri Inc.,Hui&Yuri Inc.,165 1/2,Hudson Street,Manhattan,10014,"165 1/2 Hudson Street, Manhattan, NY",4610756,10.0,16.0,160.0,NaN,NaN,NaN,yes,no,yes,0.0,RW,yes,yes,yes,07/15/2020 12:51:00 PM,NaN,NaN,NaN,NaN,NaN,undefined,NaN,NaN,Sushi Teru Hudson Street,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14259,2834,EC884462-3EC9-4A26-832B-A48241B581D8,Both,Kon'To Euro Caribbean Cuisine Corp.,Kon'To Euro Caribbean Cuisine Corp.,Seawalk Restaurant,261 267,Dyckman,Manhattan,10034,"261 267 Dyckman, Manhattan, NY",50064881,107.0,4.0,428.0,58.0,8.0,464.0,yes,yes,yes,1308802.0,OP,no,NaN,yes,06/21/2020 12:25:00 PM,NaN,NaN,NaN,NaN,NaN,undefined,NaN,NaN,Kon'To Euro Caribbean Cuisine Corp. Dyckman,,
14277,3465,F7F1FA77-A43C-471D-8AE3-D1DBF7E16757,Both,Liebman's Delicatessen& Restuarant,Py Deli Bros Inc.,Py Deli Bros Inc.,552,235th,Bronx,10463,"552 235th, Bronx, NY",40986189,60.0,12.0,720.0,60.0,6.0,360.0,yes,yes,yes,1202885.0,RW,no,NaN,yes,06/22/2020 12:19:00 PM,NaN,NaN,NaN,NaN,NaN,undefined,NaN,NaN,Liebman's Delicatessen& Restuarant 235th,,
14288,1710,D433561D-BD82-4AC9-8133-D6B1B469C271,Both,Henry's End,Bridge 72 LLC,Henry's End,72,72nd Henry Street,Brooklyn,11201,"72 72nd Henry Street, Brooklyn, NY",50089785,20.0,10.0,200.0,6.0,4.0,24.0,yes,yes,yes,1316528.0,OP,yes,yes,yes,06/19/2020 06:21:00 PM,NaN,NaN,NaN,NaN,NaN,undefined,NaN,NaN,Henry's End 72nd Henry Street,,
14417,972,31F36CEE-4CAF-4B95-81F5-4C70D131F3EC,Both,Kum Sung Bbq,New Kum Sung Inc.,Kum Sung Bbq,4114042,4007th 149th Place,Queens,11354,"4114042 4007th 149th Place, Queens, NY",50071063,18

In [11]:
fuzz.ratio("Love And Dough Pearl Street", "Love And Dough Jay Street")

88

In [12]:
inspections_df=pd.read_csv('cleaned_current_open_inspections.csv')

In [13]:
inspections_df

,Borough,RestaurantName,SeatingChoice,LegalBusinessName,Street,RestaurantInspectionID,IsSidewayCompliant,IsRoadwayCompliant,SkippedReason,InspectedOn,AgencyCode,Postcode,Latitude,Longitude,CommunityBoard,CouncilDistrict,CensusTract,BIN,BBL,NTA,Building Number,Business Address
0,Manhattan,Oscar Wilde,Both,Camelot Castle LLC,West 27th Street,72891,NaN,Non-Compliant,NaN,12/20/2021 04:06:58 PM,DOT,10001,40.744876,-73.989657,5.0,3.0,58.0,1015677.0,1.008290e+09,Hudson Yards-Chelsea-Flatiron-Union Square,45.0,"45 West 27th Street, Manhattan, NY"
1,Manhattan,La Rubia Restaurant,Both,La Rubia Restaurant Inc.,Broadway,72892,NaN,For HIQA Review,NaN,12/20/2021 04:18:42 PM,DOT,10031,40.825863,-73.950874,9.0,7.0,229.0,1062369.0,1.020910e+09,Hamilton Heights,3517.0,"3517 Broadway, Manhattan, NY"
2,Manhattan,Thai Sliders,Sidewalk,Silom Thai Inc.,8th Avenue,72893,NaN,Non-Compliant,NaN,12/20/2021 04:35:41 PM,DOT,10011,40.741906,-74.000945,4.0,3.0,81.0,1013845.0,1.007670e+09,Hudson Yards-Chelsea-Flatiron-Union Square,150.0,"150 8th Avenue, Manhattan, NY"
3,Brooklyn,Otway,Both,St James 930 LLC,Fulton Street,72894,NaN,Cease and Desist,NaN,12/20/2021 04:38:45 PM,DOT,11238,40.682833,-73.963833,2.0,35.0,201.0,3335112.0,3.020130e+09,Clinton Hill,930.0,"930 Fulton Street, Brooklyn, NY"
4,Brooklyn,Williamsburg Thai Cuisine,Both,Williamsburg Thai Cuisine Ny Inc.,Bedford Avenue,72896,NaN,Compliant,NaN,12/20/2021 04:52:41 PM,DOT,11249,40.716913,-73.958728,1.0,33.0,553.0,3062192.0,3.023350e+09,North Side-South Side,212.0,"212 Bedford Avenue, Brooklyn, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79198,Brooklyn,Sunshine Co,Both,780 Washington LLC,Wahington Avenue,107873,NaN,Pre-Removal,NaN,12/01/2023 03:44:49 PM,DOT,11238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780.0,"780 Wahington Avenue, Brooklyn, NY"
79199,Manhattan,Moustache,Sidewalk,Bedford Pitza Corp.,7th Avenue South,107874,NaN,Cease and Desist,NaN,12/01/2023 04:13:28 PM,DOT,10014,40.730812,-74.004395,2.0,3.0,67.0,1087324.0,1.005860e+09,West Village,29.0,"29 7th Avenue South, Manhattan, NY"
79200,Brooklyn,Mekelburg's,Both,"Hop, Stock & Barrel Ii LLC",Kent Avenue,107875,NaN,Skipped Inspection,No Seating,12/01/2023 04:42:51 PM,DOT,11238,40.713757,-73.967200,1.0,33.0,551.0,3424711.0,3.024280e+09,North Side-South Side,319.0,"319 Kent Avenue, Brooklyn, NY"
79201,Brooklyn,Sunday In Brooklyn,Both,Sunday In Brooklyn LLC,Wythe Avenue,107876,NaN,Compliant,NaN,12/01/2023 04:48:20 PM,DOT,11249,40.714171,-73.965208,1.0,33.0,551.0,3321284.0,3.024150e+09,North Side-South Side,348.0,"348 Wythe Avenue, Brooklyn, NY"


In [15]:
dohmh_df=pd.read_csv('cleaned_dohmh.csv')

In [16]:
dohmh_df

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1,Business Address
0,50106271,Sour Mouse,Manhattan,110,Delancey Street,10002,6462567220,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.718640,-73.988481,103.0,1.0,1800.0,1087565.0,1.004100e+09,Chinatown,NaN,"110 Delancey Street, Manhattan, NY"
1,50117820,Modello Bar - Barclays Center,Brooklyn,620,Atlantic Avenue,11217,9174174384,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,Park Slope-Gowanus,NaN,"620 Atlantic Avenue, Brooklyn, NY"
2,50141790,Nan,Manhattan,30,East 20th Street,10003,9176670036,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.738929,-73.989221,105.0,2.0,5200.0,1016181.0,1.008480e+09,Hudson Yards-Chelsea-Flatiron-Union Square,NaN,"30 East 20th Street, Manhattan, NY"
3,50134862,Fine And Raw Chocolate,Brooklyn,70,Scott Avenue,11237,6462440734,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,East Williamsburg,NaN,"70 Scott Avenue, Brooklyn, NY"
4,50127238,The Deluxe Party Ktv,Queens,3420,Linden Pl,11354,9172957843,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.766275,-73.831338,407.0,20.0,86900.0,4112007.0,4.049500e+09,Flushing,NaN,"3420 Linden Pl, Queens, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209767,50129459,Blueberry,Brooklyn,1849,Coney Island Avenue,11230,3473719836,Bakery Products/Desserts,08/03/2023,Violations were cited in the following area(s).,02A,Time/Temperature Control for Safety (TCS) food...,Critical,50.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.611980,-73.962744,314.0,48.0,54200.0,3181708.0,3.067580e+09,Midwood,NaN,"1849 Coney Island Avenue, Brooklyn, NY"
209768,41444866,Pies-N-Thighs,Brooklyn,166,South 4th Street,11211,3475296090,American,08/09/2023,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage associated w...,Critical,18.0,B,08/09/2023,12/01/2023,Cycle Inspection / Re-inspection,40.711289,-73.961300,301.0,34.0,52300.0,3063458.0,3.024460e+09,North Side-South Side,NaN,"166 South 4th Street, Brooklyn, NY"
209769,50123805,Qk Restaurant Inc.,Queens,4626,Kissena Boulevard,11355,7189990806,Chinese,08/01/2022,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,29.0,C,08/01/2022,12/01/2023,Pre-permit (Operational) / Re-inspection,40.750040,-73.818520,407.0,20.0,84500.0,4314742.0,4.051490e+09,Flushing,NaN,"4626 Kissena Boulevard, Queens, NY"
209770,50056510,Ginbo's Hamburger House,Bronx,118,East 170th Street,10452,3479634644,Hamburgers,09/28/2022,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,21.0,NaN,NaN,12/01/2023,Cycle Inspection / Compliance Inspection,40.839304,-73.915334,204.0,16.0,22102.0,2008084.0,2.028420e+09,West Concourse,NaN,"118 East 170th Street, Bronx, NY"


In [ ]:
#I'm not sure if OLAP database is the correct option for this since we can't cleanly combine all 3.